# VacationPy

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [ ]:
weather_df=pd.read_csv('../output/cities.csv')

weather_df

In [ ]:
#Check types to prevent problems with data like in weatherpy.
print(weather_df.dtypes)

In [ ]:
#Change humidity
weather_df["Humidity"] = weather_df["Humidity"].astype(float)

### Humidity Heatmap

Configure gmaps.Use the Lat and Lng as locations and Humidity as the weight.Add Heatmap layer to map.

In [ ]:
#latitude and longitude are locations
location = weather_df[["Lat", "Lng"]]

# Get humidity
hum_weight = weather_df["Humidity"]

In [ ]:
#Choose type of google maps
fig = gmaps.figure(map_type="HYBRID")

# Create the heat layer to go on top of map 
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False,max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


### Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.Drop any rows will null values.

In [ ]:
nice_weather = pd.DataFrame(weather_df[weather_df["Max Temp"].between(65, 85) & (weather_df['Wind Speed'] <= 15) & 
                                       (weather_df["Cloudiness"] <30)& (weather_df["Humidity"]<= 80)]) 
nice_weather.describe()

## Hotel Map
Store into variable named hotel_df.Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.

In [ ]:
target_type = "lodging"
coords = location
radius = 5000

# rewrite params dict

params = {
    "location": coords,
    "types": target_type,
    "radius": radius,
    "key": gkey
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print("Searching for an hotel nearby")

# Run request
response = requests.get(base_url, params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

hotel_name = response.json()
 

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure